# Monitoring Models with AML

Application Insights is an application performance management service in Azure that enables:
- capture
- storage
- analysis of telemetry data from applications

You can use App Insights in to monitor telemetry from any kind of apps, both running in azure and not running in azure. All that is needed is **a low-overhead instrumentation package** that captures & sends telemtry and data to Application Insights. 

# Enabling App Insights

How to log telemtry from an AML WS?  
You must have App Insights resource associated with AML ws, you have to configure your aml service to use it for telemtry logging.

How to connect AML ws and App Insights?  

When creating an AML ws, you can select an azure app insight resourse to associate with the ws. You can find it by viewing the overview page on the ws blade in the portal, or by using the **get_details()** method of the **Workspace** object.

```python
from azureml.core import Workspace

ws = Workspace.from_config()
ws.get_details()['applicationInsights']
```

## Enabling App Insights for a deployed service.

When deploying a new real-time inferencing service, you can enable App Insights in the deployment config for the service.
```python
def_config = AciWebservice.deploy_config(
    cpu_cores=1,
    memory_gb=1,
    enable_app_insights=True
)
```

What if the service is already deployed?
No worries, you can still enable an App Insights, by modifying the deployment config for the AKS in the azure portal. Or, you can update any web service by using the SDK:
```python
service = ws.webservice['my-svc']
service.update(enable_app_insights=True)
```

## Capturing and Viewing telemetry

App insights automaticall capture any info written to standard outputs and error logs, and provides a way to query to allow you to view these logs.

#### Writing log data
You can write any values to standard output log in the scoring script, by using the **print** statement, as shown below:

```python
def init()
    global model
    model = joblib.load(Model.get_model_path('my_model'))
def run(raw_data):
    data = json.loads(raw_data)['data']
    predictions = model.predict(data)
    log_txt = 'Data:' + str(data) + ' - Predictions:' str(predictions)
    print(log_txt)
    return predictions.tolist()
```

AML creates custom dimension in the App Insights data model for the output you write.

#### Query logs in App Insights
To query the data, you can:
- use Log Analytics query interface for App Insights in the portal
    - provides SQL like query syntax
    
example query to return the **timestamp** and **customDimension.Content** fields from log traces that have a message field valye of STDOUT and customDimensions.
```SQL
---SQL Language
traces
|where message =='STDOUT'
  and customDimensions.['Service Name'] = 'my-svc'
| project timestamp, customDimension.Content